## Chapter 5. Advanced Computer Vision
### Transfer learning
We are going to try and re-purpose a pre-trained ImageNet nn on the CIFAR-10 dataset. We'll be using pytorch to show two
different techniques, **feature extraction** and **fine tuning**.
#### Feature extraction
Feature extraction involves modifying the final layer of the existing net to accommodate the new task. Every weight of
the network, except for the final, modified layer, are locked and will not change during the training procedure. This is 
shown below in the ```t1_feature_extractor()``` function.

#### Fine tuning
Fine tuning involves modifying the final layer of the existing net to accommodate the new task and then training the whole
network, starting from the pre-trained weights. This is shown below in the ```t1_fine_tuning()``` function.

In [1]:
# TODO: comment everything
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import models, transforms
import os
print(os.getcwd())
batch_size = 50
# training data
train_data_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_data_transform)

from torch.utils.data.dataloader import  DataLoader

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=4)

val_data_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=val_data_transform)

val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=4)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def train_model(model: nn.Module, loss_function: nn.CrossEntropyLoss, optimizer: torch.optim.Adam, data_loader: torch.utils.data.DataLoader):
    model.train()
    
    current_loss = 0.0
    current_acc = 0.0
    
    # iterate
    for i, (inputs, labels) in enumerate(data_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        with torch.set_grad_enabled(True):
            outputs = model(inputs)
            _, predictions = torch.max(outputs, 1)
            loss = loss_function(outputs, labels)
            
            loss.backward()
            optimizer.step()
            
        current_loss += loss.item() * inputs.size(0)
        hits = predictions == labels.data
        current_acc += torch.sum(hits)
        
    total_loss = current_loss / len(data_loader.dataset)
    total_acc = current_acc / len(data_loader.dataset)
    
    print('train loss: {:.4f}; accuracy: {:.4f}'.format(total_loss, total_acc))
    
    
def test_model(model: nn.Module, loss_function: nn.CrossEntropyLoss, data_loader: torch.utils.data.DataLoader):
    model.eval()
    
    current_loss = 0.0
    current_acc = 0.0
    
    # iterate
    for i, (inputs, labels) in enumerate(data_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            _, predictions = torch.max(outputs, 1)
            loss = loss_function(outputs, labels)
            
        current_loss += loss.item() * inputs.size(0)
        hits = predictions == labels.data
        current_acc += torch.sum(hits)
        
    total_loss = current_loss / len(data_loader.dataset)
    total_acc = current_acc / len(data_loader.dataset)
    print('test loss: {:.4f}; accuracy: {:.4f}'.format(total_loss, total_acc))
    
    
def t1_feature_extractor(epochs: int = 3):
    print('\n********** BEGINNING FEATURE EXTRACTION PROCEDURE **********\n')
    model: nn.Module = torchvision.models.resnet18(pretrained=True)
    
    for param in model.parameters():
        param.requires_grad = False
        
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, 10)
    
    model = model.to(device)
    
    loss_function = nn.CrossEntropyLoss()
    
    optimizer = optim.Adam(model.fc.parameters())
    
    for epoch in range(epochs):
        print('Epoch {}/{}'.format(epoch + 1, epochs))
        
        train_model(model, loss_function, optimizer, train_loader)
        test_model(model, loss_function, val_loader)
    
    print('\n********** FEATURE EXTRACTION PROCEDURE COMPLETED **********\n')
    
    
def t1_fine_tuning(epochs: int = 3):
    print('\n********** BEGINNING FINE TUNING PROCEDURE **********\n')
    model: nn.Module = models.resnet18(pretrained=True)
        
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, 10)
    
    model = model.to(device)
    
    loss_function = nn.CrossEntropyLoss()
    
    optimizer = optim.Adam(model.parameters())
    
    for epoch in range(epochs):
        print('epoch: {}/{}'.format(epoch + 1, epochs))
        
        train_model(model, loss_function, optimizer, train_loader)
        test_model(model, loss_function, val_loader)
        
    print('\n********** FINE TUNING PROCEDURE COMPLETED **********\n')

# EXECUTE
t1_feature_extractor(5)
t1_fine_tuning(5)

C:\Users\Luca\PycharmProjects\deeplearningstuff\Python_deep_learning_bk
Files already downloaded and verified
Files already downloaded and verified

********** BEGINNING FEATURE EXTRACTION PROCEDURE **********

Epoch 1/5
train loss: 1.0433; accuracy: 0.6446
test loss: 0.8522; accuracy: 0.7046
Epoch 2/5
train loss: 0.8525; accuracy: 0.7029
test loss: 0.8256; accuracy: 0.7110
Epoch 3/5
train loss: 0.8311; accuracy: 0.7100
test loss: 0.8349; accuracy: 0.7046
Epoch 4/5
train loss: 0.8193; accuracy: 0.7124
test loss: 0.8167; accuracy: 0.7152
Epoch 5/5
train loss: 0.8171; accuracy: 0.7152
test loss: 0.7839; accuracy: 0.7223

********** FEATURE EXTRATION PROCEDURE COMPLETED **********


********** BEGINNING FINE TUNING PROCEDURE **********

epoch: 1/5
train loss: 0.8025; accuracy: 0.7194
test loss: 0.7358; accuracy: 0.7384
epoch: 2/5
train loss: 0.5268; accuracy: 0.8188
test loss: 0.5423; accuracy: 0.8189
epoch: 3/5
train loss: 0.4305; accuracy: 0.8508
test loss: 0.4316; accuracy: 0.8576
epoc

pag 125